In [1]:
import numpy as np
import pandas as pd
import jieba
import json
import jieba.analyse
from tqdm import tqdm
import re

In [2]:
data = pd.read_csv(r'C:\Users\JDD\工作文件\近30日文章内容.csv')
test=pd.read_csv(r'C:\Users\JDD\工作文件\test_set_1225.csv',encoding='gbk')
test_data=test[['itemid','c_title','c_content']]
data=pd.concat([data,test_data])
test_list=list(test['itemid'].unique())
data=data.reset_index(drop=True)
data

,itemid,c_title,c_content
0,2130495,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】,"[{""type"":""1"",""content"":""下一个视频应该就是飙车剪辑了""}]"
1,3609545,阳光明媚的一天,"[{""isEmptyForward"":false,""content"":"""",""id"":"""",..."
2,1636262,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用,"[{""type"":""1"",""content"":""卖400万的摩托会飞？没错！它就是一台飞行摩..."
3,3594525,警钟长鸣 又一江西少年 过弯死亡！,"[{""content"":""关键词 超速驾驶 坠入路边菜地 不幸身亡"",""type"":""..."
4,869674,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动,"[{""content"":""铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动..."
...,...,...,...
65150,3818063,高级定制改装哈雷机车,NaN
65151,3817180,BWM定制改装K1600B，够用科幻了吧，收藏了,"[{""content"":""这个改装车叫STELTH CROW。是以宝马K1600B为平台的改..."
65152,3817944,川崎中国发布ninja400/z400的召回信息。,"[{""content"":""几天前川崎在海外发布了400cc摩托车的召回信息，今天中午国内川崎..."
65153,3818304,张雪实测，凯越Cobra321R极速150km/h+,"[{""duration"":""73"",""a"":false,""b"":false,""img"":""h..."


### 1 提取Title+内容

In [3]:
#单个样本测试
#json.loads(data.c_content[0])[0].get('content')

In [9]:
def extractContent(data):
    if data not in ('', np.nan, None):
        content_list = json.loads(data,strict=False)
        content=''
        if content_list not in ('', np.nan, None):
            for item in content_list:
                str_item = item.get('content')
                if str_item is not None:
                    content = content + str_item
        else:
            content=data
        return content
def clean_data(data):
    data=re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', data, flags=re.MULTILINE)
    data = data.replace('\n', '').replace('\xa0', '').replace('\u3000', ' ').replace('\u200B', ' ').replace('\r', '').replace('\t', '').replace('/', '，').replace('丨', '，').replace('！', '。').replace('？', '。').replace('：', '，').replace('~', '。').replace('.', '。').replace(',', '，').replace('-', '').lower()
    return data

In [4]:
data['content'] = data.c_content.apply(lambda x: extractContent(x))
data

,itemid,c_title,c_content,content
0,2130495,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】,"[{""type"":""1"",""content"":""下一个视频应该就是飙车剪辑了""}]",下一个视频应该就是飙车剪辑了
1,3609545,阳光明媚的一天,"[{""isEmptyForward"":false,""content"":"""",""id"":"""",...",
2,1636262,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用,"[{""type"":""1"",""content"":""卖400万的摩托会飞？没错！它就是一台飞行摩...",卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用Ludovic Lazaret这个...
3,3594525,警钟长鸣 又一江西少年 过弯死亡！,"[{""content"":""关键词 超速驾驶 坠入路边菜地 不幸身亡"",""type"":""...",关键词 超速驾驶 坠入路边菜地 不幸身亡\n\n 邹某(男，23岁，家住赣州市章贡区...
4,869674,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动,"[{""content"":""铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动...",铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动\n转自头条视频/摩托车之家\n
...,...,...,...,...
65150,3818063,高级定制改装哈雷机车,NaN,None
65151,3817180,BWM定制改装K1600B，够用科幻了吧，收藏了,"[{""content"":""这个改装车叫STELTH CROW。是以宝马K1600B为平台的改...",这个改装车叫STELTH CROW。是以宝马K1600B为平台的改装定制车。\n虽然说是以1...
65152,3817944,川崎中国发布ninja400/z400的召回信息。,"[{""content"":""几天前川崎在海外发布了400cc摩托车的召回信息，今天中午国内川崎...",几天前川崎在海外发布了400cc摩托车的召回信息，今天中午国内川崎官方也正式发布了召回通知。...
65153,3818304,张雪实测，凯越Cobra321R极速150km/h+,"[{""duration"":""73"",""a"":false,""b"":false,""img"":""h...",


In [5]:
data['content'].fillna('。',inplace=True)
data

,itemid,c_title,c_content,content
0,2130495,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】,"[{""type"":""1"",""content"":""下一个视频应该就是飙车剪辑了""}]",下一个视频应该就是飙车剪辑了
1,3609545,阳光明媚的一天,"[{""isEmptyForward"":false,""content"":"""",""id"":"""",...",
2,1636262,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用,"[{""type"":""1"",""content"":""卖400万的摩托会飞？没错！它就是一台飞行摩...",卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用Ludovic Lazaret这个...
3,3594525,警钟长鸣 又一江西少年 过弯死亡！,"[{""content"":""关键词 超速驾驶 坠入路边菜地 不幸身亡"",""type"":""...",关键词 超速驾驶 坠入路边菜地 不幸身亡\n\n 邹某(男，23岁，家住赣州市章贡区...
4,869674,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动,"[{""content"":""铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动...",铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动\n转自头条视频/摩托车之家\n
...,...,...,...,...
65150,3818063,高级定制改装哈雷机车,NaN,。
65151,3817180,BWM定制改装K1600B，够用科幻了吧，收藏了,"[{""content"":""这个改装车叫STELTH CROW。是以宝马K1600B为平台的改...",这个改装车叫STELTH CROW。是以宝马K1600B为平台的改装定制车。\n虽然说是以1...
65152,3817944,川崎中国发布ninja400/z400的召回信息。,"[{""content"":""几天前川崎在海外发布了400cc摩托车的召回信息，今天中午国内川崎...",几天前川崎在海外发布了400cc摩托车的召回信息，今天中午国内川崎官方也正式发布了召回通知。...
65153,3818304,张雪实测，凯越Cobra321R极速150km/h+,"[{""duration"":""73"",""a"":false,""b"":false,""img"":""h...",


In [6]:
data.drop(columns=['c_content'],inplace=True)
data

,itemid,c_title,content
0,2130495,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】,下一个视频应该就是飙车剪辑了
1,3609545,阳光明媚的一天,
2,1636262,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用Ludovic Lazaret这个...
3,3594525,警钟长鸣 又一江西少年 过弯死亡！,关键词 超速驾驶 坠入路边菜地 不幸身亡\n\n 邹某(男，23岁，家住赣州市章贡区...
4,869674,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动\n转自头条视频/摩托车之家\n
...,...,...,...
65150,3818063,高级定制改装哈雷机车,。
65151,3817180,BWM定制改装K1600B，够用科幻了吧，收藏了,这个改装车叫STELTH CROW。是以宝马K1600B为平台的改装定制车。\n虽然说是以1...
65152,3817944,川崎中国发布ninja400/z400的召回信息。,几天前川崎在海外发布了400cc摩托车的召回信息，今天中午国内川崎官方也正式发布了召回通知。...
65153,3818304,张雪实测，凯越Cobra321R极速150km/h+,


In [7]:
data['content'] = (data['c_title']+'。')*2+data['content']
data

,itemid,c_title,content
0,2130495,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】。小忍者日本佛系跑山第二弹！...
1,3609545,阳光明媚的一天,阳光明媚的一天。阳光明媚的一天。
2,1636262,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用。卖400万的摩托会飞？没错！它就...
3,3594525,警钟长鸣 又一江西少年 过弯死亡！,警钟长鸣 又一江西少年 过弯死亡！。警钟长鸣 又一江西少年 过弯死亡！。关键...
4,869674,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动。铃木SFV650改装箭牌...
...,...,...,...
65150,3818063,高级定制改装哈雷机车,高级定制改装哈雷机车。高级定制改装哈雷机车。。
65151,3817180,BWM定制改装K1600B，够用科幻了吧，收藏了,BWM定制改装K1600B，够用科幻了吧，收藏了。BWM定制改装K1600B，够用科幻了吧，...
65152,3817944,川崎中国发布ninja400/z400的召回信息。,川崎中国发布ninja400/z400的召回信息。。川崎中国发布ninja400/z400的...
65153,3818304,张雪实测，凯越Cobra321R极速150km/h+,张雪实测，凯越Cobra321R极速150km/h+。张雪实测，凯越Cobra321R极速1...


In [11]:
tqdm.pandas(desc="my bar!") 
data['content'] = data.content.progress_apply(lambda x: clean_data(x))

my bar!: 100%|██████████| 65155/65155 [00:02<00:00, 30182.66it/s]


In [9]:
data

,itemid,c_title,content
0,2130495,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】,小忍者日本佛系跑山第二弹。夜跑神户市章山。【motovlog】。小忍者日本佛系跑山第二弹。夜...
1,3609545,阳光明媚的一天,阳光明媚的一天。阳光明媚的一天。
2,1636262,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用,卖400万的摩托会飞。没错。它就是一台飞行摩托车，空陆两用。卖400万的摩托会飞。没错。它就...
3,3594525,警钟长鸣 又一江西少年 过弯死亡！,警钟长鸣又一江西少年过弯死亡。。警钟长鸣又一江西少年过弯死亡。。关键词超速驾驶坠入路边菜地不...
4,869674,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动,铃木sfv650改装箭牌排气，声浪听着怎么样。有没有想驾驭它的冲动。铃木sfv650改装箭牌...
...,...,...,...
65150,3818063,高级定制改装哈雷机车,高级定制改装哈雷机车。高级定制改装哈雷机车。。
65151,3817180,BWM定制改装K1600B，够用科幻了吧，收藏了,bwm定制改装k1600b，够用科幻了吧，收藏了。bwm定制改装k1600b，够用科幻了吧，...
65152,3817944,川崎中国发布ninja400/z400的召回信息。,川崎中国发布ninja400，z400的召回信息。。川崎中国发布ninja400，z400的...
65153,3818304,张雪实测，凯越Cobra321R极速150km/h+,张雪实测，凯越cobra321r极速150km，h+。张雪实测，凯越cobra321r极速1...


In [6]:
#保存data['content'] = data.content.apply(lambda x: clean_data(x))数据
#data.to_csv(r'C:\Users\JDD\工作文件\articles_recent30days_title2.csv',encoding='utf-8_sig')
#data=pd.read_csv(r'C:\Users\JDD\工作文件\articles_recent30days_title2.csv',encoding='utf-8_sig')

## 2 数据预处理

In [2]:
import numpy as np
import pandas as pd
import jieba
import json
import jieba.analyse
from collections import Counter
import math

In [8]:
#data = pd.read_csv('./key_words.csv')

In [9]:
#data = data.loc[data.content.notnull(),:]
#data = data.reset_index(drop=True)

In [33]:
car_type_tb=pd.read_csv(r'F:\work\Downloads\全部车型.csv',encoding='utf-8')
new_name_list_kw=[]
new_name_list_q=[]
for i in car_type_tb.index:
    goods_name=car_type_tb['goods_name'][i].split(' ')
    keywords=car_type_tb['keywords'][i]
    for nm in goods_name:
        if '版' in nm or '款' in nm and len(goods_name)>1:
            goods_name.remove(nm)
    goods_name=' '.join(goods_name)
    origin_name_list=[]
    origin_name_list.append(goods_name)
    if type(keywords)==str:
        for k in keywords.split(','):
            onm=k.split(' ')
            for nm in onm:
                if '版' in nm or '款' in nm and len(onm)>1:
                    onm.remove(nm)
            k=' '.join(onm)
            origin_name_list.append(k)
    for name in origin_name_list:   #kw部分
        name=name.replace("'","")
        nonboard_goods_name=name.replace(' ','').replace('/','').replace('-','').replace('·','').replace('.','')
        lower_goods_name1=nonboard_goods_name.lower()
        lower_goods_name2=name.lower()
        tmp_name_lst=list(set([#name,
                               #nonboard_goods_name,
                               lower_goods_name1
                               #lower_goods_name2
                                ]))
        for j in tmp_name_lst:
            if j not in new_name_list_kw:
                new_name_list_kw.append(j)
    for name in origin_name_list:   #q部分
        lower_goods_name=name.lower()
        chinese = re.findall(".*?([\u4E00-\u9FA5]+).*?",lower_goods_name)
        eng=re.findall('[a-zA-Z0-9]+',lower_goods_name)
        eng=[x for x in eng if len(x)>2]
        tmp_name_lst=eng+chinese
        for j in tmp_name_lst:
            if j not in new_name_list_q:
                new_name_list_q.append(j)

In [34]:
new_name_list_kw

['tricker250',
 '魔术师',
 '雅马哈魔术师',
 'byq150t5f',
 'byq150',
 '比亚乔150',
 '台风150',
 '500nouvofalcom',
 '200elite',
 'gpz750turboprototype',
 'dragster800rramerica',
 'scramblerurbanenduro',
 '杜卡迪scrambler',
 '杜卡迪自游',
 '杜卡迪攀爬者',
 'scrambler',
 'c1econcept',
 'c3x132hellcat',
 'db6delirio1000',
 'combiz125',
 'dynasuperglidelowrider',
 'xciting300i',
 '光阳300',
 '光阳赛艇',
 '赛艇300',
 '巡戈300',
 'xaxisconcept',
 'sport1000classicprototype',
 'xv1700roadstarwarrior',
 'wr250z',
 'giantschopper',
 'b120wraith',
 '250california',
 'xvs1300vstar',
 'flhtcuse8electraglideultraclassiccvo110thanniversary',
 '跑跑dy100t8',
 'superbike1198',
 '1198',
 'r25',
 'xv1700roadstarsilverados',
 '175sport',
 '雅梦二代hj125t5c',
 '悦胜k3qm1253kb',
 'intelchopper',
 'vf700finterceptor',
 '天啸zs15040f',
 '宗申150',
 '宗申zs150',
 'bj250gy2',
 '贝纳利250',
 '贝纳利bj250',
 '贝纳利滑胎',
 '贝纳利越野',
 'vfr750fp',
 'diaveltitanium',
 'vf400f',
 '250dianamark3',
 'f41000s',
 'vs125ehj125ta22',
 '豪爵铃木125',
 '豪爵vs125',
 'xvs250v',
 '格格lh110t5',
 '格

In [ ]:
'''
car_type=pd.read_csv(r'C:\Users\JDD\工作文件\车型.csv',encoding='gbk')
with open(r'C:\Users\JDD\工作文件\user_dict.txt', 'a',encoding="utf-8") as f:
    for i in car_type.index:
        goods_name=car_type['goods_name'][i]
        nonboard_goods_name=goods_name.replace(' ','').replace('/','').replace('-','').replace('·','').replace('.','')
        lower_goods_name1=nonboard_goods_name.lower()
        lower_goods_name2=goods_name.lower()
        name_lst=list(set([goods_name,nonboard_goods_name,lower_goods_name1,lower_goods_name2]))
        for i in name_lst:
            f.write(i)
            f.write('\n')
f.close()
'''

In [3]:
#设置停用词
jieba.analyse.set_stop_words(r'C:\Users\JDD\工作文件\stopwords.txt')

In [14]:
#设置userdict
jieba.load_userdict(r'F:\work\Downloads\user_dict_kw.txt')

In [5]:
#设置idf path
jieba.analyse.set_idf_path(r'C:\Users\JDD\Documents\WXWork\1688852691841887\Cache\File\2021-04\idf.txt')

In [6]:
#设置stop_words
stop_words = set((
        "the", "of", "is", "and", "to", "in", "that", "we", "for", "an", "are",
        "by", "be", "as", "on", "with", "can", "if", "from", "which", "you", "it",
        "this", "then", "at", "have", "all", "not", "one", "has", "or", "that"
    ))
stop_content = open(r'C:\Users\JDD\工作文件\stopwords.txt', 'rb').read().decode('utf-8')
for line in stop_content.splitlines():
    stop_words.add(line)

In [7]:
allowPOS=['a','ad','ag','b','i','j','l','m','n','nr','ns','nt','nx','nz','s','r','t','tg','v','vd','vg','vn','x','z','eng']
#提取tf-idf关键词
def tf_idf_(data):
    tfidf_lst=jieba.analyse.extract_tags(data,topK=200,withWeight=True,allowPOS=allowPOS)
    tfidf_dct=dict(tfidf_lst)
    if len(tfidf_lst)>0:
        max_value=np.max(list(tfidf_dct.values()))
        tfidf_dct={x:math.sqrt(y/max_value) for x,y in tfidf_dct.items()}
    return json.dumps(tfidf_dct,ensure_ascii=False)

#提取textrank关键词
def textrank_(data):
    txtrk=jieba.analyse.TextRank() 
    txtrk.span=10
    return json.dumps(dict(txtrk.textrank(data,topK=200,withWeight=True,allowPOS=allowPOS)),ensure_ascii=False)


#提取最终关键词
def extract_tokens(data):
    '''
    tfidf_keywords = tf_idf_(data)
    textrank_keywords = textrank_(data)
    tfidf_dict = dict(tfidf_keywords)
    textrank_dict = dict(textrank_keywords)
    key_words=tfidf_dict.keys()&textrank_dict.keys()
    keywords_dict ={}
    for word in key_words:
        keywords_dict[word] = tfidf_dict[word]*textrank_dict[word]
    tokens = sorted(keywords_dict.keys(),key=lambda x: x[1], reverse=True)
    '''
    tokens=list(dict(data).keys())
    return tokens

#提取带权重的关键词
def extract_keywords_weight(data):
    tfidf_dict = tf_idf_(data)
    textrank_keywords = textrank_(data)
    textrank_dict = dict(textrank_keywords)
    if len(tfidf_dict.keys())>0:
        max_value=np.max(list(tfidf_dict.values()))
        tfidf_dict={x:y/max_value for x,y in tfidf_dict.items()}
    if len(textrank_keywords)>0:
        max_value=np.max(list(textrank_dict.values()))
        textrank_dict={x:y/max_value for x,y in textrank_dict.items()}
    key_words=tfidf_dict.keys()&textrank_dict.keys()
    keywords_dict ={}
    for word in key_words:
        if word in tfidf_dict.keys():
            tfidf_value=tfidf_dict[word]
        else:
            tfidf_value=0
        if word in textrank_dict.keys():
            textrank_value=textrank_dict[word]
        else:
            textrank_value=0
        keywords_dict[word] = tfidf_value*textrank_value
    keywords_weight = sorted(keywords_dict.items(),key=lambda x: x[1], reverse=True)
    return keywords_weight

def get_keywords_weight(a,b):
    a=eval(a)
    b=eval(b)
    if len(a.keys())>0:
        max_value=np.max(list(a.values()))
        a={x:y/max_value for x,y in a.items()}
    if len(b.keys())>0:
        max_value=np.max(list(b.values()))
        b={x:y/max_value for x,y in b.items()}
    key_words=a.keys()&b.keys()
    keywords_dict ={}
    for word in key_words:
        if word in a.keys():
            tfidf_value=a[word]
        else:
            tfidf_value=0
        if word in b.keys():
            textrank_value=b[word]
        else:
            textrank_value=0
        keywords_dict[word] = tfidf_value*textrank_value
    keywords_weight = dict(sorted(keywords_dict.items(),key=lambda x: x[1], reverse=True))
    return json.dumps(keywords_weight,ensure_ascii=False)

In [118]:
string='[{"content":"https://youtu.be/hhqOHvWx6Lk\n021年Aprilia RSV4测试Max Biaggi","highlightPositionList":[],"img":"","type":"1"}]'
x=extractContent(string)
stcs='2021 Aprilia RSV4 Sound Exhaust Test'
stc=(stcs+' ')*2+x
stc=clean_data(stc)
print(stc)
print('tfidf:',tf_idf_(stc))
print('textrank:',textrank_(stc))
get_keywords_weight(tf_idf_(stc),textrank_(stc))

2021 aprilia rsv4 sound exhaust test 2021 aprilia rsv4 sound exhaust test 021年aprilia rsv4测试max biaggi
tfidf: {"aprilia": 1.0, "sound": 0.9886485491952772, "hau": 0.9669392772050115, "test": 0.8839384110655715, "2021": 0.7206889064999226, "st": 0.6923121282722776, "021": 0.6849809948139607, "gi": 0.653899443434861, "bi": 0.636546897654267, "ax": 0.534849749644405, "ag": 0.531317999181672, "测试": 0.48322654364808354}
textrank: {"aprilia": 1.0, "hau": 0.6737137486318338, "test": 0.6513896973750704, "st": 0.6398283756800179, "sound": 0.6304395591162018, "2021": 0.483305147756332, "测试": 0.4756933895283215, "021": 0.4372464303749625, "ax": 0.42692739716276584, "bi": 0.380557132243768, "gi": 0.32806079268338273, "ag": 0.3255055188083498}


'{"aprilia": 1.0, "hau": 0.6514402851451442, "sound": 0.623283155475543, "test": 0.5757883740822032, "st": 0.44296094449602763, "2021": 0.3483126584422944, "021": 0.29950549485709504, "bi": 0.24224246190997514, "测试": 0.22986767245801226, "ax": 0.22834201148884278, "gi": 0.21451876974846326, "ag": 0.17294694097584454}'

In [131]:
string='[{"content":"-","highlightPositionList":[],"img":"","type":"1"}]'
stcs='Ducati scrambler800 【viva ride】'
x=extractContent(string)
stc=stcs*2+x
stc=clean_data(stc)
print(stc)
print('tfidf:',tf_idf_(stc))
print('textrank:',textrank_(stc))
get_keywords_weight(tf_idf_(stc),textrank_(stc))

ducati scrambler800 【viva ride】ducati scrambler800 【viva ride】
tfidf: {"viva": 1.0, "ride": 0.7896991233891882}
textrank: {"viva": 1.0, "ride": 0.9961264494011038}


'{"viva": 1.0, "ride": 0.7866401838768363}'

In [16]:
stcs='诺马500'
stcs=clean_data(stcs)
print(stcs)
print('tfidf:',tf_idf_(stcs))
print('textrank:',textrank_(stcs))

诺马500
tfidf: {"诺马500": 1.0}
textrank: {}


In [25]:
txtrk=jieba.analyse.TextRank() 
textrank_(stcs)

'{"风格": 1.0, "杜卡迪自由": 0.9980043667706294, "摩托车": 0.9958996297784177, "复古": 0.9936798241818273, "喜欢": 0.9913386590222457, "博洛尼亚": 0.9888694994162014}'

In [19]:
data=data[:100]

In [21]:
%%time
data['tfidf权重'] = data.content.apply(lambda x: tf_idf_(x))

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Wall time: 1.55 s


In [22]:
%%time
data['textrank权重'] = data.content.apply(lambda x: textrank_(x))

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Wall time: 2.07 s


In [26]:
%%time
data['最终关键词权重']=data.apply(lambda x:get_keywords_weight(x['tfidf权重'],x['textrank权重']),axis=1)

Wall time: 28.9 ms


In [22]:
%%time
data['tokens'] = data['最终关键词权重'].progress_apply(lambda x: extract_tokens(x))

my bar!: 100%|██████████| 65155/65155 [00:00<00:00, 205452.43it/s]


Wall time: 323 ms


In [33]:
data

AttributeError: 'DataFrame' object has no attribute 'type'

In [27]:
data=data.reset_index(drop=True)
data

,itemid,c_title,content,tfidf权重,textrank权重,最终关键词权重
0,2130495,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】,小忍者日本佛系跑山第二弹。夜跑神户市章山。【motovlog】。小忍者日本佛系跑山第二弹。夜...,"{""神户"": 1.0, ""市章"": 1.0, ""motovlog"": 1.0, ""佛系"": ...","{""市章"": 1.0, ""神户"": 0.9802435136470803, ""motovlo...","{""市章"": 1.0, ""神户"": 0.9802435136470803, ""motovlo..."
1,3609545,阳光明媚的一天,阳光明媚的一天。阳光明媚的一天。,"{""阳光明媚"": 1.0}","{""阳光明媚"": 1.0, ""一天"": 0.9935371392126439}","{""阳光明媚"": 1.0}"
2,1636262,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用,卖400万的摩托会飞。没错。它就是一台飞行摩托车，空陆两用。卖400万的摩托会飞。没错。它就...,"{""飞行"": 1.0, ""空陆"": 0.5529265162969731, ""喷气"": 0....","{""飞行"": 1.0, ""这个"": 0.6718162627212947, ""车型"": 0....","{""飞行"": 1.0, ""车型"": 0.21058919373179172, ""品牌"": 0..."
3,3594525,警钟长鸣 又一江西少年 过弯死亡！,警钟长鸣又一江西少年过弯死亡。。警钟长鸣又一江西少年过弯死亡。。关键词超速驾驶坠入路边菜地不...,"{""邹某"": 1.0, ""超速"": 0.885542798602541, ""死亡"": 0.8...","{""驾驶"": 1.0, ""邹某"": 0.734773964807324, ""机车"": 0.7...","{""邹某"": 0.734773964807324, ""驾驶"": 0.650018564971..."
4,869674,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动,铃木sfv650改装箭牌排气，声浪听着怎么样。有没有想驾驭它的冲动。铃木sfv650改装箭牌...,"{""箭牌"": 1.0, ""冲动"": 0.8683996340867756, ""有没有"": 0...","{""有没有"": 1.0, ""驾驭"": 0.9962337111643115, ""怎么样"": ...","{""箭牌"": 0.7610797457301912, ""冲动"": 0.74827149753..."
...,...,...,...,...,...,...
95,993261,美式重机车！加速更快，性能更卓绝，新款人体工程学摩托车,美式重机车。加速更快，性能更卓绝，新款人体工程学摩托车。美式重机车。加速更快，性能更卓绝，新...,"{""judge"": 1.0, ""卓绝"": 0.6030226891555273, ""胜利"":...","{""judge"": 1.0, ""摩托车"": 0.8648771404046748, ""巡航""...","{""judge"": 1.0, ""巡航"": 0.34913820510401306, ""胜利""..."
96,971550,体验贝纳利帝国400的长途感受。,体验贝纳利帝国400的长途感受。。体验贝纳利帝国400的长途感受。。厦门太阳不大热，这种天气...,"{""龙岩"": 1.0, ""帝国"": 0.9539392169277288, ""长汀"": 0....","{""感受"": 1.0, ""水箱"": 0.8578250703108831, ""帝国"": 0....","{""龙岩"": 0.7826052348245601, ""帝国"": 0.77448270178..."
97,2113852,韩国“正宗”美式太子~轻骑GV到底什么来头？「杨一睡前分享」,韩国“正宗”美式太子。轻骑gv到底什么来头。「杨一睡前分享」。韩国“正宗”美式太子。轻骑gv...,"{""轻骑"": 1.0, ""韩国"": 0.9603008329714576, ""kr"": 0....","{""轻骑"": 1.0, ""韩国"": 0.7636760104983903, ""太子"": 0....","{""轻骑"": 1.0, ""韩国"": 0.7333587090019238, ""太子"": 0...."
98,1784069,本田思域挑战CBR1000RRCBR1000RR：让你十秒,本田思域挑战cbr1000rrcbr1000rr，让你十秒。本田思域挑战cbr1000rrc...,"{""cbr1000rr"": 1.0, ""思域"": 0.7489014085231174, ""...","{""cbr1000rr"": 1.0, ""本田"": 0.5223544715983707, ""...","{""cbr1000rr"": 1.0, ""思域"": 0.3895248992660717, ""..."


In [55]:
data.to_csv(r'C:\Users\JDD\工作文件\key_words1231_限定词性_窗口10_lower_nonurl_平滑.csv',encoding='utf-8_sig',index=False)

In [ ]:
data=pd.read_csv(r'C:\Users\JDD\工作文件\key_words_限定词性_title2.csv',encoding='utf-8_sig')

## 3 计算文章向量

In [24]:
import pandas as pd
import numpy as np
import math
import random
import re
import numexpr as ne 
from operator import itemgetter

In [25]:
word_vectors=pd.read_csv(r'F:\work\12.3 文章关键词\word2vec\model_size100_mincount50_title3_lower_nonurl_ud.csv')
vec_dict={}
for i in word_vectors.index:
    word=word_vectors['word'][i]
    tmp_list=[]
    for j in range(1,101):
        value=word_vectors[str(j)][i]
        tmp_list.append(value)
    vec_dict[word]=np.array(tmp_list)

In [26]:
essay_content=pd.read_csv(r'F:\work\12.3 文章关键词\essay_content.csv')
essay_info={}
for i in essay_content.index:
    itemid=essay_content['itemid'][i]
    title=essay_content['c_title'][i]
    content=essay_content['c_content'][i]
    essay_info.setdefault(itemid,{})
    essay_info[itemid]['title']=title
    essay_info[itemid]['content']=content
essay_content=pd.read_csv(r'C:\Users\JDD\工作文件\test_set_1225.csv',encoding='gbk')
for i in essay_content.index:
    itemid=essay_content['itemid'][i]
    title=essay_content['c_title'][i]
    content=essay_content['c_content'][i]
    essay_info.setdefault(itemid,{})
    essay_info[itemid]['title']=title
    essay_info[itemid]['content']=content

In [27]:
def capital_to_lower(dict_info):
    new_dict = {}
    for i, j in dict_info.items():
        new_dict[i.lower()] = j
    return new_dict

In [28]:
key_dict={}
for i in data.index:
    itemid=data['itemid'][i]
    weights=capital_to_lower(dict(data['最终关键词权重'][i]))
    key_dict[itemid]=weights

In [29]:
for essay in essay_info.keys():
    essay_info[essay]['keywords']=[]
    if essay in key_dict.keys():
        essay_info[essay]['keywords']=list(key_dict[essay].keys())

In [30]:
item_vec_dict={}
for item in key_dict.keys():
    for key in key_dict[item].keys():
        if key.lower() in vec_dict.keys():
            item_vec_dict.setdefault(item,np.zeros([100,]))
            item_vec_dict[item]+=vec_dict[key.lower()]*key_dict[item][key]
    if item in item_vec_dict.keys():
        x=item_vec_dict[item]
        y=np.linalg.norm(x)
        item_vec_dict[item]=list(x/y)

In [31]:
item_lst=list(item_vec_dict.keys())
vecary= np.zeros((len(item_lst),100))
for i in range(len(item_lst)):
    vecary[i]=np.array(item_vec_dict[item_lst[i]])

In [32]:
test_list=[x for x in test_list if x in item_lst]
vec=item_vec_dict[test_list[0]]
test_vecary=np.array(vec).reshape(1,100)
for i in range(1,len(test_list)):
    vec=item_vec_dict[test_list[i]]
    test_vecary=np.insert(test_vecary,i,values=vec,axis=0)

##   4 计算相似度

In [33]:
sim_matrix=np.dot(test_vecary,vecary.T)
sim_matrix

array([[0.40405695, 0.09073253, 0.32840474, ..., 0.38675266, 0.34150106,
        0.27009534],
       [0.50669926, 0.34259433, 0.38756288, ..., 0.40654638, 0.30519365,
        0.49903021],
       [0.37074635, 0.22180975, 0.36432078, ..., 0.53507123, 0.40633399,
        0.21111958],
       ...,
       [0.39767303, 0.19904971, 0.39689985, ..., 1.        , 0.4526324 ,
        0.23473152],
       [0.34969045, 0.19188803, 0.39224897, ..., 0.4526324 , 1.        ,
        0.20724609],
       [0.47608508, 0.40512077, 0.31374811, ..., 0.23473152, 0.20724609,
        1.        ]])

In [34]:
sim_matrix.shape

(49, 64486)

In [37]:
i=2
itemid=test_list[i]
title=essay_info[itemid]['title']
keywords=essay_info[itemid]['keywords']
print(title)
print(keywords)
print('------'*8)
sorted_list=sorted(enumerate(list(sim_matrix[i])), key=itemgetter(1),reverse=True)
for j,sim in sorted_list[1:11]:
    title=essay_info[item_lst[j]]['title']
    keywords=essay_info[item_lst[j]]['keywords']
    print(title)
    print(keywords)
    print(sim)
    print('------'*8)

NameError: name 'essay_info' is not defined

## 5 输出相似度

In [36]:
L=[]
for i in range(len(test_list)):
    itemid1=test_list[i]
    title1=essay_info[itemid1]['title']
    content1=essay_info[itemid1]['content']
    tokens1=essay_info[itemid1]['keywords']
    keywords1=key_dict[itemid1]
    sorted_list=sorted(enumerate(list(sim_matrix[i])), key=itemgetter(1),reverse=True)
    for j,sim in sorted_list[1:11]:
        itemid2=item_lst[j]
        title2=essay_info[itemid2]['title']
        content2=essay_info[itemid2]['content']
        tokens2=essay_info[itemid2]['keywords']
        keywords2=key_dict[itemid2]
        L.append((itemid1,title1,content1,tokens1,keywords1,itemid2,title2,content2,tokens2,keywords2,sim))
output = pd.DataFrame(L,columns=['essay_1','essay1_title','essay1_content','essay1_tokens','essay1_keywords',
                                 'essay_2','essay2_title','essay2_content','essay2_tokens','essay2_keywords','sim'])
output.sort_values(by=['essay_1','sim'],ascending=[True,False],inplace=True)

In [37]:
#output.to_csv(r'C:\Users\JDD\工作文件\测试集3_限制词性_title2_关键词&_小写_去网址_平滑.csv',index=False,encoding='utf-8_sig')
output

,essay_1,essay1_title,essay1_content,essay1_tokens,essay1_keywords,essay_2,essay2_title,essay2_content,essay2_tokens,essay2_keywords,sim
160,3814213,女骑为了冬天也能舒服的骑摩托车，花5000买了这些装备：加热坐垫加热手把等，打工人不封车！,"[{""type"":1,""content"":""为了冬天也能舒服地骑车，我买了大概5000块的骑...","[加热, 电加热, 手把, 坐垫, 装备, 不封, 工人, 冬天, 舒服, 防雾, 马甲, ...","{'加热': 1.0, '电加热': 0.9576933373591722, '手把': 0...",856070,冬季骑摩托车，有哪些既保暖又实用，又能兼顾帅气的方法和建议？,"[{""duration"":"""",""img"":""http://file.jddmoto.com...","[防寒, 防风, 手套, 电加热, 保暖, 手指, 电热, 护膝, 定制, 防水, 手把, ...","{'防寒': 1.0, '防风': 0.7940884664044737, '手套': 0....",0.901572
161,3814213,女骑为了冬天也能舒服的骑摩托车，花5000买了这些装备：加热坐垫加热手把等，打工人不封车！,"[{""type"":1,""content"":""为了冬天也能舒服地骑车，我买了大概5000块的骑...","[加热, 电加热, 手把, 坐垫, 装备, 不封, 工人, 冬天, 舒服, 防雾, 马甲, ...","{'加热': 1.0, '电加热': 0.9576933373591722, '手把': 0...",3648104,冬季骑车时，教你如何在瑟瑟发抖的人群中做最有温度的骑士！,"[{""type"":""1"",""content"":""一直以来，恶劣天气和低温对于追逐速度和激情的...","[冬季, 手套, 温度, 选择, 保暖, 电加热, 防水, 骑行服, 手把, 防风, 骑车,...","{'冬季': 1.0, '手套': 0.3254706585215903, '温度': 0....",0.884064
162,3814213,女骑为了冬天也能舒服的骑摩托车，花5000买了这些装备：加热坐垫加热手把等，打工人不封车！,"[{""type"":1,""content"":""为了冬天也能舒服地骑车，我买了大概5000块的骑...","[加热, 电加热, 手把, 坐垫, 装备, 不封, 工人, 冬天, 舒服, 防雾, 马甲, ...","{'加热': 1.0, '电加热': 0.9576933373591722, '手把': 0...",3259529,冬天出行也不怕，实用摩托骑行必备物件，你都有哪些？,"[{""type"":""1"",""content"":""马上就即将迎来冬季；在北方的小伙伴，如果你家...","[冬季, 护具, 防风, 骑行, 选择, 手套, 把手, 护罩, 防水, 物件, 实用, 防...","{'冬季': 0.7958327939945405, '护具': 0.78094521733...",0.879328
163,3814213,女骑为了冬天也能舒服的骑摩托车，花5000买了这些装备：加热坐垫加热手把等，打工人不封车！,"[{""type"":1,""content"":""为了冬天也能舒服地骑车，我买了大概5000块的骑...","[加热, 电加热, 手把, 坐垫, 装备, 不封, 工人, 冬天, 舒服, 防雾, 马甲, ...","{'加热': 1.0, '电加热': 0.9576933373591722, '手把': 0...",1329048,温暖出行~冬季骑行装备分享~手套篇~,"[{""content"":""近似气温骤降，虽不及东北的“嘎嘎冷”\n但真到了数九寒天。骑士心中...","[手套, 电加热, 骑行, 灵活, 保暖, 温度, 价格, 适宜, 防护性, 挡车, 冬季,...","{'手套': 1.0, '电加热': 0.34458371487153117, '骑行': ...",0.876528
164,3814213,女骑为了冬天也能舒服的骑摩托车，花5000买了这些装备：加热坐垫加热手把等，打工人不封车！,"[{""type"":1,""content"":""为了冬天也能舒服地骑车，我买了大概5000块的骑...","[加热, 电加热, 手把, 坐垫, 装备, 不封, 工人, 冬天, 舒服, 防雾, 马甲, ...","{'加热': 1.0, '电加热': 0.9576933373591722, '手把': 0...",2216589,摩旅攻略 | 冬季湿冷骑行，如何 “享受” 雨雪冰的 “折磨”？,"[{""type"":""1"",""content"":""康庄大道谁不爱？"",""img"":"""",""li...","[保暖, 加热, 天气, 骑行, 雨雪, 夹克, 确保, 手套, 双手, 阿正, 装备, 寒...","{'保暖': 0.7919513164774518, '加热': 0.67352034032...",0.864519
...,...,...,...,...,...,...,...,...,...,...,...
125,3818721,本田中排门店名单出炉，NSS350\CM300发布定价区间！,"[{""content"":""「杨一的摩托分享146」12月24日圣诞前夜，平安夜傍晚，HOND...","[中排, 本田, 市场, 定价, cm300, hondawing, 门店, nss350,...","{'中排': 0.7815153545437532, '本田': 0.61552681113...",3252424,买不起大排就不配有梦想?,"[{""type"":""1"",""content"":""前两天，北京车展。本田中国亮相了两款车——N...","[本田, 中国, 市场, nss350, cm300, 车友们, 叫法, 生产, 两款, 配...","{'本田': 1.0, '中国': 0.4199511214198561, '市场': 0....",0.902566
126,3818721,本田中排门店名单出炉，NSS350\CM300发布定价区间！,"[{""content"":""「杨一的摩托分享146」12月24日圣诞前夜，平安夜傍晚，HOND...","[中排, 本田, 市场, 定价, cm300, hondawing, 门店, nss350,...","{'中排': 0.7815153545437532, '本田': 0.61552681113...",3204939,本田CM300即将北京车展发布，猜猜价格？,"[{""type"":""1"",""content"":""经过几年的耕耘，本田已经在国内高端车型市场扎...","[cm300, 本田, 佛沙, 定价, 车型, pm, 国内, 发布, 价格, 两款, 北京...","{'cm300': 0.8672447707855766, '本田': 0.61812246...",0.900553
127,3818721,本田中排门店名单出炉，NSS350\CM300发布定价区间！,"[{""content"":""「杨一的摩托分享146」12月24日圣诞前夜，平安夜傍晚，HOND...","[中排, 本田, 市场, 定价, cm300, hondawing, 门店, nss350,...","{'中排': 0.7815153545437532, '本田': 0.61552681113...",3401767,"不出手则已,出手必伤人——本田国产中排量车型战略启动后会出手打谁","[{""type"":""1"",""content"":""我是骠骑府，同名公号创始人及作者，摩托车评论...","[本田, 国内, 车型, 国产品牌, cc, 排量, 市场, 品牌, 进口, 生产, 排量车...","{'本田': 1.0, '国内': 0.2601089734396761, '车型': 0....",0.898195
128,3818721,本田中排门店名单出炉，NSS350\CM300发布定价区间！,"[{""content"":""「杨一的摩托分享146」12月24日圣诞前夜，平安夜傍晚，HOND...","[中排, 本田, 市场, 定价, cm300, hondawing, 门店, nss350,...","{'中排': 0.7815153545437532, '本田': 0.61552681113...",3232730,脱离五羊、新大洲，本田佛沙升级NSS350亮相，售价多少呢？,"[{""type"":""1"",""content"":""随着国内经济水平的提高，摩托车在近几年的发展...","[款车, 本田, 国内, 亮相, nss350, 两款, 表现, 新大洲, 五羊, forz...","{'款车': 0.7102060619728325, '本田': 0.63800881026...",0.897815


## 6 用户关键词

In [38]:
import pandas as pd
import numpy as np
import re
from gensim.models import KeyedVectors
from tqdm import tqdm

In [39]:
user_data = pd.read_csv(r'C:\Users\JDD\工作文件\users30Days_1203.csv')
item_data = data

In [40]:
def drop_symbols(data):
    key_dict=dict(data)
    return key_dict

In [41]:
item_data['KeyWords_WithWeight'] = item_data['最终关键词权重'].progress_apply(lambda x: drop_symbols(x))

my bar!: 100%|██████████| 65155/65155 [00:00<00:00, 405435.10it/s]


In [42]:
essay_data = pd.concat([item_data['itemid'],pd.DataFrame(vecary, columns=['essayVectors_'+str(i) for i in range(100)])],axis=1)

In [43]:
#essay_data.to_csv(r'C:\Users\JDD\工作文件\essay_vectors1203.csv')

In [44]:
#合并用户数据和文章数据
user_data = user_data.merge(item_data,on='itemid')

In [45]:
def extract_user_keywords(data):
    a = {}
    for i in range(data.shape[0]):
        tmp=data['KeyWords_WithWeight'].iloc[i]
        for m,n in tmp.items():
            a.setdefault(m,0)
            a[m]+=1
    b = sorted(a.items(),key=lambda x: x[1], reverse=True)[0:20]
    return b

In [46]:
user_keywords = user_data.groupby('deviceid').progress_apply(lambda x: extract_user_keywords(x))
user_keywords = user_keywords.reset_index()

my bar!: 100%|██████████| 1120955/1120955 [03:20<00:00, 5578.38it/s]


In [47]:
#保存用户关键词数据
#user_keywords.to_csv(r'C:\Users\JDD\工作文件\users_keywords_1204_lower.csv',encoding='utf-8_sig')

In [48]:
def compute_user_vectors(data):
    word_vector = np.zeros([1,100],dtype=np.float32)
    for word in data[:5]:
        if word[0] in vec_dict.keys():
            word_vector+=vec_dict[word[0]]*word[1]
    return word_vector

In [49]:
user_array = np.zeros((user_keywords.shape[0],100))
for i in tqdm(range(user_keywords.shape[0])):
    user_array[i]= compute_user_vectors(user_keywords[0][i])

100%|██████████| 1120955/1120955 [01:02<00:00, 18019.86it/s]


In [50]:
%%time
user_vectors = pd.concat([user_keywords[['deviceid']],pd.DataFrame(user_array,columns=['userVectors_'+str(i) for i in range(100)])],axis=1)

Wall time: 1.85 s


In [51]:
#user_vectors.to_csv(r'C:\Users\JDD\工作文件\user_vectors1221.csv')

In [52]:
i=8
u_keywords=user_keywords[0][i]
u_name=user_keywords['deviceid'][i]
print(u_keywords)
print('------'*8)
read_essays=user_data.loc[user_data['deviceid']==u_name]['c_title'].unique()
for j in read_essays:
    print(j)

[('零关税', 2), ('进口', 2), ('日本', 2), ('每日', 2), ('资讯', 2), ('机车', 2), ('分享', 2), ('压弯', 2), ('油门', 2), ('发布', 2), ('重大新闻', 1), ('摩托车', 1), ('炸缸', 1), ('二代', 1), ('川崎', 1), ('心痛', 1), ('狂轰', 1), ('拆掉', 1), ('排气管', 1), ('17', 1)]
------------------------------------------------
【重大新闻】日本进口摩托车即将零关税？
叫你不要压弯不要压弯，你非是不听呐~
富二代拆掉排气管狂轰油门，17万川崎Z900当场炸缸，好心痛！
神仙压弯
你会不会忽然地出现，说一句好久不见？雅马哈R1或将于下周在国内发布
重大好消息！日本进口摩托即将零关税！
求交警找？！江苏女骑士误上高架还发抖音，结果.....
头盔真好看呀！
春风700-CLX用车感受，不定时更新，目前6000公里
宝马发布【2021款S1000R】，变的我都不敢看了。。。
V型双缸900，国产新车亮相：定位复古巡航，厂家成立不到1年


In [53]:
print(u_keywords)
print('------'*8)
u_vec=user_array[i]
if np.linalg.norm(u_vec)>0:
    u_vec=u_vec/np.linalg.norm(u_vec)
    sm=np.dot(u_vec,vecary.T)
    sorted_list=sorted(enumerate(list(sm)), key=itemgetter(1),reverse=True)
    for j,sim in sorted_list[1:11]:
        title=essay_info[item_lst[j]]['title']
        keywords=essay_info[item_lst[j]]['keywords']
        if title not in read_essays:
            print(title)
            print(keywords)
            print(sim)
            print('------'*8)

[('零关税', 2), ('进口', 2), ('日本', 2), ('每日', 2), ('资讯', 2), ('机车', 2), ('分享', 2), ('压弯', 2), ('油门', 2), ('发布', 2), ('重大新闻', 1), ('摩托车', 1), ('炸缸', 1), ('二代', 1), ('川崎', 1), ('心痛', 1), ('狂轰', 1), ('拆掉', 1), ('排气管', 1), ('17', 1)]
------------------------------------------------
日本摩托能够“零关税”,还是洗洗睡吧!
['零关税', '摩托车', '进口', '日本', '关税', '国内', '日系', '泰国', '洗洗', '冷静下来', '扬言']
0.8418164679696023
------------------------------------------------
日本摩托能够“零关税”，还是洗洗睡吧！
['零关税', '摩托车', '进口', '日本', '关税', '国内', '泰国', '日系', '冷静下来', '洗洗', '硬性', '扬言']
0.8415998323148317
------------------------------------------------
【日本摩托车零关税】RCER协定！澄清事件！
['零关税', '协定', '澄清', '事件', '日本', '摩托车', '每日', '资讯', '分享', '机车']
0.821837079183981
------------------------------------------------
RCEP签订 日本进口摩托车即将零关税？ 你别高兴的太早了
['零关税', '摩托车', '进口', '日本', 'ep', '协定', '行业', '发展', '沸腾', '本土', '新闻', '承诺', '竞争']
0.8144234739341674
------------------------------------------------
日系摩托车即将零关税？网友：高兴早了，还记得WTO关税免征吗
['零关税', '关税', '摩托车', '日系', 'ep', '国内',